In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, LongType, StringType, DoubleType
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import StringIndexer, VectorAssembler, Imputer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pyspark.sql.functions as F
from itertools import combinations
import os

### Check Python Path

In [2]:
import sys
sys.executable

'/tmp/demos/bin/python3'

In [3]:
DATA_FOLDER = "data"

NUMBER_OF_FOLDS = 3
SPLIT_SEED = 7576
TRAIN_TEST_SPLIT = 0.8

### Function for data reading

In [4]:
def read_data(spark: SparkSession) -> DataFrame:
    """
    read data; since the data has the header we let spark guess the schema
    """
    
    # Read the Titanic CSV data into a DataFrame
    titanic_data = spark.read \
        .format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(os.path.join(DATA_FOLDER,"*.csv"))

    return titanic_data

### Writing new Transformer type class : adding cross product of features

In [5]:
class PairwiseProduct(Transformer):

    def __init__(self, inputCols, outputCols):
        self.__inputCols = inputCols
        self.__outputCols = outputCols

        self._paramMap = self._params = {}

    def _transform(self, df):
        for cols, out_col in zip(self.__inputCols, self.__outputCols):
            df = df.withColumn(out_col, col(cols[0]) * col(cols[1]))
        return df

### Task 1: Add the squared of numerical features

In [6]:
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCols, HasOutputCols
from pyspark.sql.functions import col, pow
from pyspark.sql import DataFrame

class SquaredFeatures(Transformer, HasInputCols, HasOutputCols):
    def __init__(self, inputCols=None, outputCols=None):
        super(SquaredFeatures, self).__init__()
        self._setDefault(inputCols=None, outputCols=None)
        self.setParams(inputCols=inputCols, outputCols=outputCols)

    def setParams(self, inputCols=None, outputCols=None):
        return self._set(inputCols=inputCols, outputCols=outputCols)

    def _transform(self, df: DataFrame) -> DataFrame:
        input_cols = self.getInputCols()
        output_cols = self.getOutputCols()
        
        for i, col_name in enumerate(input_cols):
            squared_col_name = output_cols[i]
            df = df.withColumn(squared_col_name, pow(col(col_name), 2))
        
        return df

### The ML Pipeline

In [7]:
def pipeline(data: DataFrame):

    """
    every attribute that is numeric is non-categorical; this is questionable
    """

    numeric_features = [f.name for f in data.schema.fields if isinstance(f.dataType, DoubleType) or isinstance(f.dataType, FloatType) or isinstance(f.dataType, IntegerType) or isinstance(f.dataType, LongType)]
    string_features = [f.name for f in data.schema.fields if isinstance(f.dataType, StringType)]
    numeric_features.remove("PassengerId")
    numeric_features.remove("Survived")
    string_features.remove("Name")

    # index string features; map string to consecutive integers - it should be one hot encoding 
    name_indexed_string_columns = [f"{v}Index" for v in string_features] 
    # we must have keep so that we can impute them in the next step
    indexer = StringIndexer(inputCols=string_features, outputCols=name_indexed_string_columns, handleInvalid='keep')

    # Fill missing values; strategy can be mode, median, mean
    
    # string columns
    imputed_columns_string = [f"Imputed{v}" for v in name_indexed_string_columns]
    imputers_string = []
    for org_col_name, indexed_col_name, imputed_col_name in zip(string_features, name_indexed_string_columns, imputed_columns_string):
        # Count the number of distinct categories in the original column
        number_of_categories = data.select(F.countDistinct(org_col_name)).take(1)[0].asDict()[f'count(DISTINCT {org_col_name})']
        
        # Create an imputer for the indexed column
        # this is the value that needs to be imputed based on the keep option above
        imputer = Imputer(inputCol=indexed_col_name, outputCol=imputed_col_name, strategy = "mode", missingValue=number_of_categories)

        # Append the imputer to the list
        imputers_string.append(imputer)

    # numeric columns
    imputed_columns_numeric = [f"Imputed{v}" for v in numeric_features]
    imputer_numeric = Imputer(inputCols=numeric_features, outputCols=imputed_columns_numeric, strategy = "mean")

    # Create squared features
    squared_features = SquaredFeatures(inputCols=imputed_columns_numeric, outputCols=[f"{col}_squared" for col in imputed_columns_numeric])

    # Create all pairwise products of numeric features
    all_pairs = [v for v in combinations(imputed_columns_numeric, 2)]
    pairwise_columns = [f"{col1}_{col2}" for col1, col2 in all_pairs]
    pairwise_product = PairwiseProduct(inputCols=all_pairs, outputCols=pairwise_columns)

    # Assemble feature columns into a single feature vector
    assembler = VectorAssembler(
        inputCols=pairwise_columns + imputed_columns_numeric + [f"{col}_squared" for col in imputed_columns_numeric] + imputed_columns_string, 
        outputCol="features"
    )

    # Define a Random Forest classifier
    classifier = RandomForestClassifier(labelCol="Survived", featuresCol="features")

    # Create the pipeline
    pipeline = Pipeline(stages=[indexer, *imputers_string, imputer_numeric, squared_features, pairwise_product, assembler, classifier])
    
    # Set up the parameter grid for maximum tree depth
    paramGrid = ParamGridBuilder() \
        .addGrid(classifier.maxDepth, [2, 4, 6, 8, 10]) \
        .addGrid(classifier.numTrees, [20, 50, 100]) \
        .build()

    # Set up the cross-validator
    evaluator = BinaryClassificationEvaluator(labelCol="Survived", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    crossval = CrossValidator(
        estimator=pipeline,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator,
        numFolds=NUMBER_OF_FOLDS,
        seed=SPLIT_SEED
    )

    # Split the data into training and test sets
    train_data, test_data = data.randomSplit([TRAIN_TEST_SPLIT, 1-TRAIN_TEST_SPLIT], seed=SPLIT_SEED)

    # Train the cross-validated pipeline model
    cvModel = crossval.fit(train_data)

    # Make predictions on the test data
    predictions = cvModel.transform(test_data)

    # Evaluate the model
    auc = evaluator.evaluate(predictions)
    print(f"Area Under ROC Curve: {auc:.4f}")

    # Get the best RandomForest model
    best_model = cvModel.bestModel.stages[-1]

    # Retrieve the selected maximum tree depth
    selected_max_depth = best_model.getOrDefault(best_model.getParam("maxDepth"))
    selected_num_trees = best_model.getOrDefault(best_model.getParam("numTrees"))


    # Print the selected maximum tree depth
    print(f"Selected Maximum Tree Depth: {selected_max_depth}")
    print(f"Selected Number of Trees: {selected_num_trees}")

### Task 2

In order to extend my Cross-Validation Parameter Grid, I used a new hyperparameter called 'numTrees' and added it to the ParameterGrid. The numTrees parameter in a Random Forest classifier specifies the number of trees to be used in the forest. Increasing the number of trees usually improves the performance of the model by reducing overfitting and capturing more complex patterns in the data. After a certain point however, it doesn't improve accuracy but adds to computational cost. By tuning numTrees, we can find an optimal balance where the model generalizes well to unseen data.

### Original Results without Squared of numerical features

In [8]:
def main():
    # Create a Spark session
    spark = SparkSession.builder \
        .appName("Predict Titanic Survival") \
        .getOrCreate()

    data = read_data(spark)
    pipeline(data)

    spark.stop()
    
main()

Area Under ROC Curve: 0.8828
Selected Maximum Tree Depth: 4


### New Results

In [8]:
def main():
    # Create a Spark session
    spark = SparkSession.builder \
        .appName("Predict Titanic Survival") \
        .getOrCreate()

    data = read_data(spark)
    pipeline(data)

    spark.stop()
    
main()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 05:18:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/23 05:19:49 WARN DAGScheduler: Broadcasting large task binary with size 1209.7 KiB
24/05/23 05:20:00 WARN DAGScheduler: Broadcasting large task binary with size 1209.7 KiB
24/05/23 05:20:00 WARN DAGScheduler: Broadcasting large task binary with size 1451.1 KiB
24/05/23 05:20:00 WARN DAGScheduler: Broadcasting large task binary with size 1662.6 KiB
24/05/23 05:20:01 WARN DAGScheduler: Broadcasting large task binary with size 1220.3 KiB
24/05/23 05:20:33 WARN DAGScheduler: Broadcasting large task binary with size 1200.3 KiB
24/05/23 05:20:43 WARN DAGScheduler: Broadcasting large task binary with size 1200.3 KiB
24/05/23 05:20:43 WARN DAGScheduler: Broadcasting large task binary with size 1429.7 KiB
24/05/23 05:20:

Area Under ROC Curve: 0.8912
Selected Maximum Tree Depth: 6
Selected Number of Trees: 50
